In [2]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

## Import models from Hugging Face hub for every supported task
This sample notebook shows how to import and register models from [HuggingFace hub](https://huggingface.co/models) for every supported task. 

### How does import work?
The import process runs as a job in your AzureML workspace using components from the `azureml` system registry. The models are downloaded and converted to MLflow packaged format. You can then register the models to your AzureML Workspace or Registry that makes them available for inference or fine tuning. 

### What models are supported for import?
Any model from Hugging Face hub can be downloaded using the `download_model` component. Following table shows the task supported for mlflow along with the corresponding Huggingface model id for each task:

|Task |Hugging Face Model ID|
|:---|:---|
|fill-mask|bert-base-cased|
|token-classification|Jean-Baptiste/camembert-ner|
|question-answering|deepset-minilm-uncased-squad2|
|summarization|facebook/bart-large-cnn|
|text-generation|distilgpt2|
|text-classification|microsoft/deberta-base-mnli|
|translation|t5-small|
|image-classification|microsoft/resnet-50|
|text-to-image|runwayml/stable-diffusion-v1-5|

Conversion to MLflow will fail if you attempt to download a model that has a task type other than the above with error - `Exception: Unsupported task {task name}`

### Why convert to MLflow?
MLflow is AzureML's recommended model packaging format. 
* **Inference benefits**: AzureML supports no-code-deployment for models packaged as MLflow that enables a seamless inference experience for the models. Learn more about [MLflow model packaging](https://learn.microsoft.com/en-us/azure/machine-learning/concept-mlflow-models) and [no-code-deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=sdk). 
* **Fine tuning benefits**: Foundation models imported and converted to MLflow format can be fine tuned using AzureML's fine tuning pipelines. You can use the no-code UI wizards, or the code based job submission with the SDK or CLI/YAML. AzureML's fine tuning pipelines are built using components. This gives you the flexibility to compose your own fine tuning pipelines containing your own jobs for data transformation, post processing and the AzureML fine tuning components. Learn more about pipelines using [sdk](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python) or [CLI](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipelines-cli).

### What happens if I just download model and register models without converting to MLflow? That's because the task of the model I'm interested in is not among the supported list of tasks.
You can still download and register the model using the outputs of the `download_model` job. You need to [write your own inference code](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=python) in this case. It also means that fine tuning is not yet supported if the task type of the model you are interested in is not in the supported list.

### Outline
* Setup pre-requisites such as compute.
* Pick a model to import.
* Configure the import job.
* Run the fine tuning job.
* Register the fine tuned model. 


**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](https://aka.ms/azureml-workspace-configuration)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](https://aka.ms/azureml-sdkv2-install) - check the getting started section


**Motivations** - This notebook explains how to create model importing/publishing pipeline job in workspace using pipeline component registered in a registry